<a href="https://colab.research.google.com/github/devikaajay/Startup_Success_Prediction/blob/main/startupfinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [111]:

import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.impute import SimpleImputer
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time


In [112]:
import pandas as pd
import numpy as np

# 1. Load the CSV file
try:
    df = pd.read_csv('/content/big_startup_secsees_dataset.csv', encoding='utf-8')
except FileNotFoundError:
    print("Error: 'big_startup_secsees_dataset.csv' not found. Please upload your file and ensure the name is correct.")
    df = pd.DataFrame()

REQUIRED_COLUMNS = [
    'name',
    'homepage_url',
    'category_list',
    'funding_total_usd',
    'status',
    'country_code',
    'state_code',
    'region',
    'city',
    'funding_rounds'
]

# --- Data Cleaning and Selection ---

if df.empty:
    print("DataFrame is empty. Skipping further processing.")
else:

    missing_cols = [col for col in REQUIRED_COLUMNS if col not in df.columns]
    if missing_cols:
        print(f"Error: The following required columns are missing from the uploaded file: {', '.join(missing_cols)}")

        df = df[[col for col in REQUIRED_COLUMNS if col in df.columns]]
    else:
        df = df[REQUIRED_COLUMNS]


    print(f"Original shape after column selection: {df.shape}")

    df_no_all_duplicates = df.drop_duplicates()
    print(f"Shape after removing all-column duplicates: {df_no_all_duplicates.shape}")

    df_unique_names = df_no_all_duplicates.drop_duplicates(subset=['name'], keep='first')
    print(f"Shape after enforcing unique company names: {df_unique_names.shape}")

    df_cleaned = df_unique_names.dropna()
    print(f"Shape after removing all rows with null values: {df_cleaned.shape}")


    print("\nFirst 5 rows of the cleaned data:")
    print(df_cleaned.head())

    CLEANED_FILE_NAME = 'cleaned_startup_data_for_prediction.csv'
    df_cleaned.to_csv(CLEANED_FILE_NAME, index=False)
    print(f"\nCleaned data saved to: {CLEANED_FILE_NAME}")

Original shape after column selection: (66368, 10)
Shape after removing all-column duplicates: (66367, 10)
Shape after enforcing unique company names: (66103, 10)
Shape after removing all rows with null values: (52551, 10)

First 5 rows of the cleaned data:
                     name                       homepage_url  \
0                   #fame                 http://livfame.com   
1                :Qounter             http://www.qounter.com   
3                 0-6.com                 http://www.0-6.com   
4        004 Technologies  http://004gmbh.de/en/004-interact   
6  Ondine Biomedical Inc.               http://ondinebio.com   

                                       category_list funding_total_usd  \
0                                              Media          10000000   
1  Application Platforms|Real Time|Social Network...            700000   
3                                        Curated Web           2000000   
4                                           Software         

In [113]:
df_cleaned.shape

(52551, 10)

In [114]:
import pandas as pd
import re
import numpy as np

# Install tabulate if it's missing (needed for to_markdown)
try:
    import tabulate
except ImportError:
    !pip install tabulate

print(f"Starting shape for advanced cleaning: {df_cleaned.shape}")


initial_shape = df_cleaned.shape[0]
name_mask = df_cleaned['name'].astype(str).str.match(r'^[a-zA-Z]')
df_cleaned = df_cleaned[name_mask]
print(f"Shape after removing names starting with non-letters: {df_cleaned.shape} ({initial_shape - df_cleaned.shape[0]} rows removed)")

initial_shape = df_cleaned.shape[0]
numeric_state_mask = df_cleaned['state_code'].astype(str).str.isnumeric()

df_cleaned = df_cleaned[~numeric_state_mask]
print(f"Shape after removing numeric state codes: {df_cleaned.shape} ({initial_shape - df_cleaned.shape[0]} rows removed)")

initial_shape = df_cleaned.shape[0]
single_letter_mask = df_cleaned['name'].astype(str).str.len() == 1

df_cleaned = df_cleaned[~single_letter_mask]
print(f"Shape after removing single-letter names: {df_cleaned.shape} ({initial_shape - df_cleaned.shape[0]} rows removed)")


initial_shape = df_cleaned.shape[0]
df_cleaned['funding_total_usd'] = pd.to_numeric(
    df_cleaned['funding_total_usd'],
    errors='coerce'
)

df_cleaned.dropna(subset=['funding_total_usd'], inplace=True)
print(f"Shape after validating/cleaning funding values: {df_cleaned.shape} ({initial_shape - df_cleaned.shape[0]} rows removed)")

TARGET_ROWS = 3000
initial_shape_trunc = df_cleaned.shape[0]

if initial_shape_trunc > TARGET_ROWS:
    df_cleaned = df_cleaned.head(TARGET_ROWS)
    print(f"Dataset truncated to the top {TARGET_ROWS} rows.")
elif initial_shape_trunc < TARGET_ROWS:
    print(f"Warning: Only {initial_shape_trunc} rows remain after cleaning. No truncation performed.")
else:
    print(f"Dataset size is exactly {TARGET_ROWS} rows after cleaning. No truncation needed.")


print("\n--- Final Cleaned Data Summary ---")
print(f"Final shape of the super-cleaned DataFrame: {df_cleaned.shape}")

print("\nFirst 5 rows of the super-cleaned data:")
print(df_cleaned.head().to_markdown(index=False, numalign="left", stralign="left"))

SUPER_CLEANED_FILE_NAME = 'super_cleaned_startup_data_for_prediction.csv'
df_cleaned.to_csv(SUPER_CLEANED_FILE_NAME, index=False)
print(f"\nSuper-cleaned data saved to: {SUPER_CLEANED_FILE_NAME}")

Starting shape for advanced cleaning: (52551, 10)
Shape after removing names starting with non-letters: (52093, 10) (458 rows removed)
Shape after removing numeric state codes: (39606, 10) (12487 rows removed)
Shape after removing single-letter names: (39606, 10) (0 rows removed)
Shape after validating/cleaning funding values: (34069, 10) (5537 rows removed)
Dataset truncated to the top 3000 rows.

--- Final Cleaned Data Summary ---
Final shape of the super-cleaned DataFrame: (3000, 10)

First 5 rows of the super-cleaned data:
| name                   | homepage_url               | category_list                                            | funding_total_usd   | status    | country_code   | state_code   | region      | city          | funding_rounds   |
|:-----------------------|:---------------------------|:---------------------------------------------------------|:--------------------|:----------|:---------------|:-------------|:------------|:--------------|:-----------------|
| Ondin

In [115]:
df_cleaned.shape

(3000, 10)

In [116]:
import requests
import pandas as pd
import time
import numpy as np
import json
import re
from bs4 import BeautifulSoup
import os

TARGET_URL = 'https://www.cbinsights.com/research-unicorn-companies'
SCRAPED_FILE_NAME = 'super_cleaned_startup_data_for_prediction.csv'


INR_CONVERSION_RATE = 88.78

SCRAPED_COLUMNS = [
    'name',
    'homepage_url',
    'category_list',
    'funding_total_usd',
    'status',
    'country_code',
    'state_code',
    'region',
    'city',
    'funding_rounds'
]

scraped_data = []

def clean_funding_value(funding_str):
    """
    Converts money strings (e.g., '$1.2B', '$500M') to numerical INR.
    The final value is multiplied by the INR_CONVERSION_RATE.
    """
    if not isinstance(funding_str, str):
        return None

    text = funding_str.strip().replace('$', '').replace(',', '').upper()

    if not text:
        return None

    usd_value = None

    if 'B' in text:
        try:
            usd_value = float(text.replace('B', '').strip()) * 1_000_000_000
        except ValueError:
            pass

    elif 'M' in text:
        try:
            usd_value = float(text.replace('M', '').strip()) * 1_000_000
        except ValueError:
            pass

    elif 'K' in text:
        try:
            usd_value = float(text.replace('K', '').strip()) * 1_000
        except ValueError:
            pass

    else:
        try:
            usd_value = float(text)
        except ValueError:
            pass

    if usd_value is not None:

        return usd_value * INR_CONVERSION_RATE

    return None


def scrape_data(url):
    """
    Fetches the HTML page and scrapes the main table elements directly.
    """
    print(f"Attempting to fetch HTML and parse the main company table from: {url}...")

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
        'Referer': 'https://www.google.com/',
    }

    try:
        response = requests.get(url, headers=headers, timeout=30)
        response.raise_for_status()

        soup = BeautifulSoup(response.content, 'html.parser')


        main_content_div = soup.find('div', id='company_list_table')
        if not main_content_div:
             main_content_div = soup

        table = main_content_div.find('table')

        if not table:
            print("ERROR: Could not find the main company table using generic selectors.")
            return []

        tbody = table.find('tbody')
        if not tbody:
            print("ERROR: Found table but could not locate the tbody for rows.")
            return []

        rows = tbody.find_all('tr')

        if not rows:
            print("ERROR: Found the table but no company rows were present.")
            return []

        print(f"SUCCESS: Found {len(rows)} company rows. This might be the full list or the initial static content.")

        page_results = []
        for i, row in enumerate(rows):

            cols = row.find_all(['td', 'th'])

            if len(cols) < 7:
                continue
            name_cell = cols[0].find('a')
            company_name = name_cell.text.strip() if name_cell else cols[0].text.strip()


            valuation_raw = cols[1].text.strip()

            category_list_raw = cols[3].text.strip()

            country_raw = cols[4].text.strip()

            city_raw = cols[5].text.strip()

            sanitized_name = re.sub(r'[^a-z0-9]', '', company_name.lower())
            homepage_url_derived = f"www.{sanitized_name}.com"
            unicorn_info = {
                'name': company_name,
                'homepage_url': homepage_url_derived,
                'category_list': category_list_raw,
                'funding_total_usd': clean_funding_value(valuation_raw),
                'status': 'operating',
                'country_code': country_raw,
                'state_code': city_raw,
                'region': city_raw,
                'city': city_raw,
                'funding_rounds': 5
            }
            page_results.append(unicorn_info)

        return page_results

    except requests.exceptions.HTTPError as e:
        print(f"ERROR: HTTP error {response.status_code} fetching HTML data {url}: {e}")
        return []
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return []


print(f"Starting web scraping process using HTML parsing for the CB Insights list.")
print(f"NOTE: Funding amounts will be converted to INR using a rate of 1 USD = {INR_CONVERSION_RATE} INR.")

scraped_data = scrape_data(TARGET_URL)

if scraped_data:
    df_new = pd.DataFrame(scraped_data, columns=SCRAPED_COLUMNS)
    df_new.replace({np.nan: None, '': None}, inplace=True)


    df_existing = pd.DataFrame()

    if os.path.exists(SCRAPED_FILE_NAME):
        try:

            df_existing = pd.read_csv(SCRAPED_FILE_NAME)
            print(f"Found {len(df_existing)} existing records in {SCRAPED_FILE_NAME}.")
        except pd.errors.EmptyDataError:
            print("Existing file found but it was empty.")
        except Exception as e:

            print(f"Error reading existing file: {e}. Starting with only new data.")

    df_existing = df_existing[SCRAPED_COLUMNS] if not df_existing.empty else df_existing
    df_combined = pd.concat([df_existing, df_new], ignore_index=True)
    initial_total = len(df_combined)

    df_combined.drop_duplicates(subset=['name'], keep='last', inplace=True)
    final_total = len(df_combined)

    df_combined.to_csv(SCRAPED_FILE_NAME, index=False)

    print("\n--- Scraping Complete ---")

    print(f"Successfully scraped {len(df_new)} new/updated records.")
    print(f"Combined and saved {final_total} unique records (total) to {SCRAPED_FILE_NAME}.")
    print(f"Total duplicates removed/overwritten: {initial_total - final_total}.")

    print("\n--- Sample of FINAL Saved Data (Funding in INR) ---")
    print(df_combined.tail().to_markdown(index=False))

else:
    print("No data was scraped. The website likely loads the table content using JavaScript that our script cannot execute. The manual copy-paste method remains the only guaranteed alternative.")


Starting web scraping process using HTML parsing for the CB Insights list.
NOTE: Funding amounts will be converted to INR using a rate of 1 USD = 88.78 INR.
Attempting to fetch HTML and parse the main company table from: https://www.cbinsights.com/research-unicorn-companies...
SUCCESS: Found 1303 company rows. This might be the full list or the initial static content.
Found 3000 existing records in super_cleaned_startup_data_for_prediction.csv.

--- Scraping Complete ---
Successfully scraped 1303 new/updated records.
Combined and saved 4279 unique records (total) to super_cleaned_startup_data_for_prediction.csv.
Total duplicates removed/overwritten: 24.

--- Sample of FINAL Saved Data (Funding in INR) ---
| name                | homepage_url               | category_list   |   funding_total_usd | status    | country_code   | state_code            | region                | city                  |   funding_rounds |
|:--------------------|:---------------------------|:----------------|--

In [117]:
df_combined.shape

(4279, 10)

In [118]:
import pandas as pd
import re
import numpy as np

FILE_TO_LOAD = 'super_cleaned_startup_data_for_prediction.csv'

try:
    df_cleaned = pd.read_csv(FILE_TO_LOAD)
except FileNotFoundError:
    print(f"Error: The file '{FILE_TO_LOAD}' was not found. Please run 'cb_insights_scraper.py' first.")
    exit()
except Exception as e:
    print(f"Error loading {FILE_TO_LOAD}: {e}")
    exit()

print(f"Starting shape for advanced cleaning: {df_cleaned.shape}")

initial_shape = df_cleaned.shape[0]
name_mask = df_cleaned['name'].astype(str).str.match(r'^[a-zA-Z]')
df_cleaned = df_cleaned[name_mask]
print(f"Shape after removing names starting with non-letters: {df_cleaned.shape} ({initial_shape - df_cleaned.shape[0]} rows removed)")


initial_shape = df_cleaned.shape[0]
numeric_state_mask = df_cleaned['state_code'].astype(str).str.isnumeric()

df_cleaned = df_cleaned[~numeric_state_mask]
print(f"Shape after removing numeric state codes: {df_cleaned.shape} ({initial_shape - df_cleaned.shape[0]} rows removed)")


initial_shape = df_cleaned.shape[0]
single_letter_mask = df_cleaned['name'].astype(str).str.len() == 1
df_cleaned = df_cleaned[~single_letter_mask]
print(f"Shape after removing single-letter names: {df_cleaned.shape} ({initial_shape - df_cleaned.shape[0]} rows removed)")



initial_shape = df_cleaned.shape[0]
df_cleaned['funding_total_usd'] = pd.to_numeric(
    df_cleaned['funding_total_usd'],
    errors='coerce'
)
# Now drop the rows that became NaN due to the coercion
df_cleaned.dropna(subset=['funding_total_usd'], inplace=True)
print(f"Shape after validating/cleaning funding values: {df_cleaned.shape} ({initial_shape - df_cleaned.shape[0]} rows removed)")


# --- 5. Reduce dataset to the 2000 most 'valid' (first) rows ---
TARGET_ROWS = 3000
initial_shape_trunc = df_cleaned.shape[0]

if initial_shape_trunc > TARGET_ROWS:
    df_cleaned = df_cleaned.head(TARGET_ROWS)
    print(f"Dataset truncated to the top {TARGET_ROWS} rows.")
elif initial_shape_trunc < TARGET_ROWS:
    print(f"Warning: Only {initial_shape_trunc} rows remain after cleaning. No truncation performed.")
else:
    print(f"Dataset size is exactly {TARGET_ROWS} rows after cleaning. No truncation needed.")


# --- Final Output and Save ---

print("\n--- Final Cleaned Data Summary ---")
print(f"Final shape of the super-cleaned DataFrame: {df_cleaned.shape}")

# Display the first few rows of the super-cleaned data
print("\nFirst 5 rows of the super-cleaned data:")
print(df_cleaned.head().to_markdown(index=False, numalign="left", stralign="left"))

# Save the super-cleaned DataFrame to a new CSV file
SUPER_CLEANED_FILE_NAME = 'super_cleaned_startup_data_for_prediction.csv'
# NOTE: This line is redundant since the file has the same name as the input,
# but we keep it here to ensure the final, cleaned data is saved.
df_cleaned.to_csv(SUPER_CLEANED_FILE_NAME, index=False)
print(f"\nSuper-cleaned data saved to: {SUPER_CLEANED_FILE_NAME}")


Starting shape for advanced cleaning: (4279, 10)
Shape after removing names starting with non-letters: (4268, 10) (11 rows removed)
Shape after removing numeric state codes: (4268, 10) (0 rows removed)
Shape after removing single-letter names: (4268, 10) (0 rows removed)
Shape after validating/cleaning funding values: (4268, 10) (0 rows removed)
Dataset truncated to the top 3000 rows.

--- Final Cleaned Data Summary ---
Final shape of the super-cleaned DataFrame: (3000, 10)

First 5 rows of the super-cleaned data:
| name                          | homepage_url               | category_list                                            | funding_total_usd   | status    | country_code   | state_code   | region      | city          | funding_rounds   |
|:------------------------------|:---------------------------|:---------------------------------------------------------|:--------------------|:----------|:---------------|:-------------|:------------|:--------------|:-----------------|
| Ondi

In [119]:
df_cleaned.shape

(3000, 10)

In [120]:
df_combined.head()

,name,homepage_url,category_list,funding_total_usd,status,country_code,state_code,region,city,funding_rounds
0,Ondine Biomedical Inc.,http://ondinebio.com,Biotechnology,7.628510e+05,operating,CAN,BC,Vancouver,Vancouver,2
2,One Inc.,http://whatis1.com,Mobile,1.150050e+06,operating,USA,CA,SF Bay Area,San Francisco,3
3,ZenChef,http://zenchef.com/en/,Local Businesses|Restaurants,1.066177e+07,operating,FRA,A8,Paris,Paris,5
4,Redox,http://www.redoxengine.com,Health Care|Health Care Information Technology...,4.000000e+06,operating,USA,WI,Madison,Madison,2
5,One Block Off the Grid (1BOG),http://1bog.org,Clean Technology|Residential Solar,5.000000e+06,closed,USA,CA,SF Bay Area,San Francisco,1


#**Data Preprocessing and EDA**

In [121]:
import pandas as pd
import numpy as np

# Define the file name used by the scraper and cleaner scripts
FILE_NAME = 'super_cleaned_startup_data_for_prediction.csv'

# --- 1. Load the Super Cleaned Data ---
try:
    df = pd.read_csv(FILE_NAME)
    print(f"Successfully loaded data for preprocessing. Initial shape: {df.shape}")
except FileNotFoundError:
    print(f"ERROR: The file '{FILE_NAME}' was not found.")
    print("Please ensure you have run 'cb_insights_scraper.py' to generate the data.")
    # Exit gracefully if the critical file is missing
    exit()
except Exception as e:
    print(f"An error occurred while reading the file: {e}")
    exit()


# --- 2. Data Preprocessing (Cleaning and Type Conversion) ---

print("\n--- 2. Data Preprocessing (Cleaning and Type Conversion) ---")

# The goal here is to make sure our columns are ready for math or grouping.

# 2.1 Type Conversion and Missing Data (NaN) Handling for Numeric Features
# We force these columns to be numbers. If the conversion fails (e.g., finding text),
# it replaces that value with NaN (Not a Number).
df['funding_total_usd'] = pd.to_numeric(df['funding_total_usd'], errors='coerce')
df['funding_rounds'] = pd.to_numeric(df['funding_rounds'], errors='coerce')

# 2.2 Missing Data Handling for Text (Categorical) Features
# For text columns, we replace any missing values (NaN) with 'Unknown'
# so they can be grouped correctly later.
df['country_code'].fillna('Unknown Country', inplace=True)
df['category_list'].fillna('Unknown Industry', inplace=True)
df['region'].fillna('Unknown Region', inplace=True)

# 2.3 Dropping Critical Missing Rows
# We must remove any rows that still have missing values in our key number columns
# (funding and rounds) because we can't do math or prediction without them.
initial_rows = df.shape[0]
df.dropna(subset=['funding_total_usd', 'funding_rounds'], inplace=True)
rows_removed = initial_rows - df.shape[0]

print(f"Rows removed due to missing Funding/Rounds: {rows_removed}")
print(f"Data shape after cleaning and preparation: {df.shape}")


# --- 3. Exploratory Data Analysis (EDA) ---

print("\n--- 3.1 Descriptive Statistics (Numeric Features in INR) ---")
# This gives us a quick summary (count, average, min, max, median) of our key numbers.
print(df[['funding_total_usd', 'funding_rounds']].describe().to_markdown(numalign="left", stralign="left"))

print("\n--- 3.2 Categorical Analysis: Top 5 Countries and Industries ---")
# This shows us which categories appear most often in the dataset.
print("Top 5 Countries:")
print(df['country_code'].value_counts().head(5).to_markdown(numalign="left", stralign="left"))
print("\nTop 5 Industries (Category List):")
print(df['category_list'].value_counts().head(5).to_markdown(numalign="left", stralign="left"))


# --- 3.3 CRITICAL STEP: Creating the Binary Target Variable (0 or 1) ---

print("\n--- 3.3 Creating Binary Target: is_highly_funded ---")
# This is our 'Success' score for prediction. We are turning a continuous number (funding)
# into a simple YES (1) or NO (0) question for the model to learn.

# 1. Calculate the middle (median) funding amount across all companies.
median_funding = df['funding_total_usd'].median()

# 2. Create the binary target column:
# 1 if the company's funding is ABOVE the median, 0 if it is at or BELOW the median.
df['is_highly_funded'] = (df['funding_total_usd'] > median_funding).astype(int)

print(f"Median Funding Total (in INR) used as threshold: ₹{median_funding:,.0f}")
print(f"Target distribution: 1 (Highly Funded) count: {df['is_highly_funded'].sum()} / Total rows: {df.shape[0]}")
print("New binary column 'is_highly_funded' added to the DataFrame.")

# --- 4. Prepare for Next Step (Feature Engineering) ---
# Save the new DataFrame with the target variable, ready for the machine learning model.
df.to_csv('prepared_for_model.csv', index=False)
print("\nPrepared data saved to: prepared_for_model.csv (Ready for Feature Engineering)")


Successfully loaded data for preprocessing. Initial shape: (3000, 10)

--- 2. Data Preprocessing (Cleaning and Type Conversion) ---
Rows removed due to missing Funding/Rounds: 0
Data shape after cleaning and preparation: (3000, 10)

--- 3.1 Descriptive Statistics (Numeric Features in INR) ---
|       | funding_total_usd   | funding_rounds   |
|:------|:--------------------|:-----------------|
| count | 3000                | 3000             |
| mean  | 2.01838e+07         | 2.14833          |
| std   | 6.6516e+07          | 1.71191          |
| min   | 100                 | 1                |
| 25%   | 669238              | 1                |
| 50%   | 3.459e+06           | 1                |
| 75%   | 1.63568e+07         | 3                |
| max   | 2.39482e+09         | 15               |

--- 3.2 Categorical Analysis: Top 5 Countries and Industries ---
Top 5 Countries:
| country_code   | count   |
|:---------------|:--------|
| USA            | 2568    |
| GBR            | 211    

/tmp/ipython-input-2445352745.py:36: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['country_code'].fillna('Unknown Country', inplace=True)
/tmp/ipython-input-2445352745.py:37: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=Tr

#**Model Building**

In [122]:
import pandas as pd
# Libraries for splitting data, training the model, and checking performance
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, f1_score
import numpy as np

# Define the file name from the previous preprocessing step
FILE_NAME = 'prepared_for_model.csv'

# --- 1. Load Data ---
try:
    df = pd.read_csv(FILE_NAME)
    print(f"Successfully loaded data for modeling. Initial shape: {df.shape}")
except FileNotFoundError:
    print(f"ERROR: The file '{FILE_NAME}' was not found.")
    print("Please ensure you have run 'data_preprocessing_eda.py' first.")
    exit()
except Exception as e:
    print(f"An error occurred while reading the file: {e}")
    exit()


# --- 1.5. Apply Domain Constraint and Rename Target ---

# Rename the target column to 'success' as requested
df.rename(columns={'is_highly_funded': 'success'}, inplace=True)

print("\n--- 1.5. Applying Domain Constraints ---")
# Constraint: A closed company cannot be considered successful (a positive prediction).
# We explicitly set the 'success' score to 0 for any row where status is 'closed'.
closed_count = df[df['status'] == 'closed'].shape[0]
df.loc[df['status'] == 'closed', 'success'] = 0
print(f"Applied constraint: Set 'success' to 0 for {closed_count} rows where status is 'closed'.")
print(f"Target column renamed to 'success'.")


# --- 2. Feature Selection ---

# X (Features): The columns the model will use to guess the outcome.
# We choose columns available to a young startup (location, industry, rounds) AND status.
feature_cols = [
    'category_list', 'country_code', 'state_code', 'region', 'city', 'funding_rounds', 'status'
]
X = df[feature_cols]

# y (Target): The column the model is trying to predict (our 0 or 1 success score).
# This is now the 'success' column.
y = df['success']

print(f"\nFeatures selected: {feature_cols}")


# --- 3. Feature Engineering (One-Hot Encoding) ---

print("\n--- 3. Feature Engineering: One-Hot Encoding (Converting Text to Numbers) ---")
# Machine learning models only understand numbers. This step creates a new column
# (with 0s and 1s) for every unique text value in the original columns.
# Example: 'country_code' turns into 'country_code_USA', 'country_code_CHN', etc.
X_encoded = pd.get_dummies(X, columns=feature_cols, drop_first=True)

print(f"Original feature columns: {len(feature_cols)}")
print(f"New encoded feature columns (numbers only): {X_encoded.shape[1]}")
print("Data is now numerical and ready for the model.")

Successfully loaded data for modeling. Initial shape: (3000, 11)

--- 1.5. Applying Domain Constraints ---
Applied constraint: Set 'success' to 0 for 216 rows where status is 'closed'.
Target column renamed to 'success'.

Features selected: ['category_list', 'country_code', 'state_code', 'region', 'city', 'funding_rounds', 'status']

--- 3. Feature Engineering: One-Hot Encoding (Converting Text to Numbers) ---
Original feature columns: 7
New encoded feature columns (numbers only): 2557
Data is now numerical and ready for the model.


In [123]:

# --- 4. Data Splitting (Train/Test Split) ---

# We split the data so the model can learn from one part (Train) and be tested
# on the other part (Test) to see if it generalizes to new data.
# Libraries for splitting data, training the model, and checking performance
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier # Added this import
from sklearn.metrics import classification_report, confusion_matrix, f1_score
import numpy as np

X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\nTraining set size: {X_train.shape[0]} rows (80% of data)")
print(f"Testing set size: {X_test.shape[0]} rows (20% of data)")



# --- 5. Hyperparameter Tuning (Finding the Best Random Forest Settings) ---

print("\n--- 5. Hyperparameter Tuning using GridSearchCV ---")
print("This step searches for the optimal parameters to maximize the F1-Score...")
#

# Define the model to tune (starting with a balanced class weight)
rf_base = RandomForestClassifier(random_state=42, class_weight='balanced')

# Define the grid of parameters to test. We try different numbers of trees (n_estimators)
# and different maximum depths of those trees (max_depth).
param_grid = {
    'n_estimators': [50, 100, 200], # Number of trees in the forest
    'max_depth': [5, 10, None] # Maximum depth of each tree (None means unlimited)
}

# Use GridSearchCV to automatically test every combination in the grid.
# We set scoring='f1' to ensure the model optimizes for the F1-Score (our primary success metric).
grid_search = GridSearchCV(
    estimator=rf_base,
    param_grid=param_grid,
    scoring='f1',
    cv=3, # Use 3-fold cross-validation
    verbose=0,
    n_jobs=-1 # Use all available cores for speed
)

# Train the grid search on the training data
grid_search.fit(X_train, y_train)

# The best model found during the search
final_model = grid_search.best_estimator_

print("Hyperparameter tuning complete.")
print(f"BEST parameters found: {grid_search.best_params_}")



print("\n--- 5. Model Training: Two Classifiers ---")

# Model A: Random Forest (Excellent for complex, mixed data)
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
rf_model.fit(X_train, y_train)
print("Random Forest Model training complete.")

# Evaluate Random Forest model
rf_y_pred = rf_model.predict(X_test)
rf_f1 = f1_score(y_test, rf_y_pred)
print(f"Random Forest F1-Score: {rf_f1:.4f}")

# Model B: K-Nearest Neighbors (KNN - Simple, distance-based model)
# We use a neighbor count of 5 as a starting point.
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train, y_train)
print("K-Nearest Neighbors (KNN) Model training complete.")

# Evaluate KNN model
knn_y_pred = knn_model.predict(X_test)
knn_f1 = f1_score(y_test, knn_y_pred)
print(f"K-Nearest Neighbors (KNN) F1-Score: {knn_f1:.4f}")


# --- 6.3 Final Model Comparison and Selection ---
print("\n--- Final Model Selection ---")
if rf_f1 >= knn_f1:
    best_model_name = "Random Forest Classifier"
    best_model = rf_model
    y_pred = rf_y_pred
    y_proba = rf_model.predict_proba(X_test)[:, 1]
    best_f1 = rf_f1
else:
    best_model_name = "K-Nearest Neighbors (KNN)"
    best_model = knn_model
    y_pred = knn_y_pred
    y_proba = knn_model.predict_proba(X_test)[:, 1]
    best_f1 = knn_f1

print(f"Based on the F1-Score, the BEST model is the: {best_model_name} (F1: {best_f1:.4f})")
print("We prioritize F1-Score over simple Accuracy because it ensures we find true winners (Recall) without making too many false positive predictions (Precision).")

# Assign the best model to 'model' for consistency with subsequent cells
model = best_model

# --- 7. Feature Importance (Why the model made the decision) ---
# Feature importance is only available for the Random Forest model.
print("\n--- 7. Feature Importance (Top 10 from Random Forest if selected) ---")
if best_model_name == "Random Forest Classifier":
    feature_importances = pd.Series(model.feature_importances_, index=X_encoded.columns)
    top_features = feature_importances.nlargest(10)
    print("Which input factors are most important for predicting 'Highly Funded' (1) (from Random Forest):")
    print(top_features.to_markdown(numalign="left", stralign="left"))
else:
    print("Feature importance is not directly available for K-Nearest Neighbors.")


Training set size: 2400 rows (80% of data)
Testing set size: 600 rows (20% of data)

--- 5. Hyperparameter Tuning using GridSearchCV ---
This step searches for the optimal parameters to maximize the F1-Score...
Hyperparameter tuning complete.
BEST parameters found: {'max_depth': None, 'n_estimators': 100}

--- 5. Model Training: Two Classifiers ---
Random Forest Model training complete.
Random Forest F1-Score: 0.7222
K-Nearest Neighbors (KNN) Model training complete.
K-Nearest Neighbors (KNN) F1-Score: 0.5731

--- Final Model Selection ---
Based on the F1-Score, the BEST model is the: Random Forest Classifier (F1: 0.7222)
We prioritize F1-Score over simple Accuracy because it ensures we find true winners (Recall) without making too many false positive predictions (Precision).

--- 7. Feature Importance (Top 10 from Random Forest if selected) ---
Which input factors are most important for predicting 'Highly Funded' (1) (from Random Forest):
|                             | 0         |
|

#F1-score, confusion matrix (for classification)

In [124]:

# 6.1 F1-Score and Classification Report
# F1-Score measures the balance between Precision (how accurate is the '1' prediction?) and
# Recall (how many true '1's did the model find?).
f1 = f1_score(y_test, y_pred)

print(f"Model F1-Score on Test Data: {f1:.4f} (Closer to 1.0 is better)")
print("\nClassification Report (Focus on F1-score for class 1):")
print(classification_report(y_test, y_pred, target_names=['Moderately Funded (0)', 'Highly Funded (1)']))


# 6.2 Confusion Matrix
# This shows exactly where the model was right and wrong on the test data.
cm = confusion_matrix(y_test, y_pred)
# Unpack the matrix components: True Negatives, False Positives, False Negatives, True Positives
tn, fp, fn, tp = cm.ravel()

print("\nConfusion Matrix (Test Data):")
print("------------------------------------------")
print("Interpreting the Matrix:")
print(f"| True Negatives (Correctly guessed 0s): {tn:<10} | False Positives (Wrongly guessed 1s): {fp:<10} |")
print(f"| False Negatives (Missed 1s): {fn:<10} | True Positives (Correctly guessed 1s): {tp:<10} |")
print("------------------------------------------")

# --- 7. Feature Importance (Why the model made the decision) ---
print("\n--- 7. Feature Importance (Top 10) ---")

# Feature importance tells us which input factors (Country, City, Round number, Industry)
# the model considered most important when making a prediction.
feature_importances = pd.Series(model.feature_importances_, index=X_encoded.columns)
top_features = feature_importances.nlargest(10)

print("Most important for predicting success (1):")
print(top_features.to_markdown(numalign="left", stralign="left"))

Model F1-Score on Test Data: 0.7222 (Closer to 1.0 is better)

Classification Report (Focus on F1-score for class 1):
                       precision    recall  f1-score   support

Moderately Funded (0)       0.76      0.79      0.77       324
    Highly Funded (1)       0.74      0.71      0.72       276

             accuracy                           0.75       600
            macro avg       0.75      0.75      0.75       600
         weighted avg       0.75      0.75      0.75       600


Confusion Matrix (Test Data):
------------------------------------------
Interpreting the Matrix:
| True Negatives (Correctly guessed 0s): 255        | False Positives (Wrongly guessed 1s): 69         |
| False Negatives (Missed 1s): 81         | True Positives (Correctly guessed 1s): 195        |
------------------------------------------

--- 7. Feature Importance (Top 10) ---
Most important for predicting success (1):
|                             | 0         |
|:----------------------------|

In [125]:
import os
import subprocess
from IPython.display import clear_output

# Create the templates directory for Flask
!mkdir -p templates
print("Created 'templates' directory.")

# --- Write requirements.txt FIRST to ensure it exists for pip install ---
with open('requirements.txt', 'w') as f:
    f.write("""Flask
pandas
scikit-learn
numpy
joblib
pyngrok
""")
print("Created requirements.txt")

# Install all dependencies including pyngrok from requirements.txt
# Adding --ignore-installed for blinker compatibility in Colab
!pip install -r requirements.txt --ignore-installed
clear_output()
print("Dependencies installed (Flask, pandas, scikit-learn, numpy, joblib, pyngrok).")

# Now that pyngrok is installed, import ngrok
from pyngrok import ngrok

# --- Write app.py (The Flask Application) ---
app_code = """
import json
import joblib
import pandas as pd
import numpy as np
from flask import Flask, request, render_template

# --- Configuration and Initialization ---
app = Flask(__name__)
MODEL_PATH = 'model.joblib'
FEATURES_PATH = 'features.json'

# Global variables to hold the loaded model and feature list
model = None
feature_names = None

# List of the original categorical columns that need OHE
OHE_COLS = ['category_list', 'country_code', 'state_code', 'region', 'city', 'status']

def load_assets():
    # Load the trained model and feature names list.
    global model, feature_names

    # 1. Load the Model
    try:
        model = joblib.load(MODEL_PATH)
        print("✅ Model loaded successfully.")
    except FileNotFoundError:
        print(f"❌ Error: Model file not found at {MODEL_PATH}. Ensure you have uploaded 'model.joblib'.")
        model = None
    except Exception as e:
        print(f"❌ Error loading model: {e}")
        model = None

    # 2. Load the Feature Names (from training data)
    try:
        with open(FEATURES_PATH, 'r') as f:
            feature_names = json.load(f)
        print(f"✅ Features list loaded successfully. Total OHE features: {len(feature_names)}")
    except FileNotFoundError:
        print(f"❌ Error: Features file not found at {FEATURES_PATH}. Ensure you have uploaded 'features.json'.")
        feature_names = []
    except Exception as e:
        print(f"❌ Error loading features: {e}")
        feature_names = []

load_assets()

@app.route('/', methods=['GET', 'POST'])
def index():
    # Renders the main form and handles prediction on POST request.

    if model is None or not feature_names:
        return render_template('index.html', error="Model assets are missing or failed to load. Please check the console output.")

    prediction_result = None

    if request.method == 'POST':
        try:
            raw_input = {
                'category_list': request.form['category_list'],
                'country_code': request.form['country_code'],
                'state_code': request.form['state_code'],
                'region': request.form['region'],
                'city': request.form['city'],
                'funding_rounds': int(request.form['funding_rounds']),
                'status': request.form['status']
            }

            input_df = pd.DataFrame([raw_input])

            input_encoded = pd.get_dummies(input_df, columns=OHE_COLS, drop_first=False)

            X_new = input_encoded.reindex(columns=feature_names, fill_value=0)

            prediction_class = model.predict(X_new)[0]

            probabilities = model.predict_proba(X_new)[0]
            class_labels = model.classes_
            prob_map = dict(zip(class_labels, probabilities))

            predicted_probability = prob_map[prediction_class] * 100

            prediction_result = {
                'class': int(prediction_class),
                'probability': f"{predicted_probability:.2f}%",
                'full_probs': {int(k): f"{v*100:.2f}%" for k, v in prob_map.items()}
            }

        except ValueError:
            return render_template('index.html',
                                   error="Invalid input. Please ensure 'Funding Rounds' is an integer and all fields are filled."
                                   )
        except Exception as e:
            print(f"An unexpected error occurred during prediction: {e}")
            return render_template('index.html',
                                   error=f"Prediction failed due to an internal error: {e}"
                                   )

    return render_template('index.html', result=prediction_result)

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)
"""
with open('app.py', 'w') as f:
    f.write(app_code)
print("Created app.py")


# --- Write templates/index.html ---
html_code = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Startup Success Prediction</title>
    <style>
        @import url('https://fonts.googleapis.com/css2?family=Inter:wght@400;600;700&display=swap');
        body {
            font-family: 'Inter', sans-serif;
            background-color: #f8f9fa;
            color: #333;
            display: flex;
            justify-content: center;
            align-items: flex-start;
            padding-top: 40px;
            margin: 0;
            min-height: 100vh;
        }
        .container {
            background-color: #fff;
            padding: 30px 40px;
            border-radius: 12px;
            box-shadow: 0 4px 15px rgba(0, 0, 0, 0.1);
            width: 100%;
            max-width: 600px;
        }
        h1 {
            text-align: center;
            color: #007bff;
            margin-bottom: 30px;
            font-weight: 700;
        }
        .form-group {
            margin-bottom: 20px;
        }
        label {
            display: block;
            margin-bottom: 8px;
            font-weight: 600;
            color: #555;
        }
        input[type="number"], input[type="text"], select {
            width: 100%;
            padding: 10px;
            border: 1px solid #ccc;
            border-radius: 8px;
            box-sizing: border-box;
            transition: border-color 0.3s;
        }
        input[type="number"]:focus, input[type="text"]:focus, select:focus {
            border-color: #007bff;
            outline: none;
        }
        button {
            width: 100%;
            padding: 12px;
            background-color: #28a745;
            color: white;
            border: none;
            border-radius: 8px;
            font-size: 16px;
            cursor: pointer;
            transition: background-color 0.3s, transform 0.1s;
            font-weight: 600;
        }
        button:hover {
            background-color: #218838;
        }
        button:active {
            transform: scale(0.99);
        }
        .result-box {
            margin-top: 30px;
            padding: 20px;
            border-radius: 8px;
            background-color: #e9f7ef;
            border: 1px solid #c3e6cb;
        }
        .result-title {
            font-size: 1.2em;
            font-weight: 700;
            color: #28a745;
            margin-bottom: 10px;
            text-align: center;
        }
        .prediction-value {
            font-size: 2em;
            font-weight: 800;
            text-align: center;
            color: #007bff;
            margin-bottom: 15px;
        }
        .prob-detail {
            font-size: 0.9em;
            color: #555;
            line-height: 1.5;
        }
        .error-message {
            margin-top: 20px;
            padding: 15px;
            background-color: #f8d7da;
            color: #721c24;
            border: 1px solid #f5c6cb;
            border-radius: 8px;
            text-align: center;
        }
    </style>
</head>
<body>
    <div class="container">
        <h1>Startup Success Prediction</h1>
        <p style="text-align: center; color: #666;">Predicts the likelihood of success (Class 1) based on key startup attributes.</p>

        {% if error %}
        <div class="error-message">{{ error }}</div>
        {% endif %}

        <form method="POST" action="/">

            <div class="form-group">
                <label for="category_list">Industry Category (e.g., Software, Biotechnology):</label>
                <input type="text" id="category_list" name="category_list" required value="Software">
            </div>

            <div class="form-group">
                <label for="country_code">Country Code (e.g., USA, GBR, CAN):</label>
                <input type="text" id="country_code" name="country_code" required value="USA">
            </div>

            <div class="form-group">
                <label for="state_code">State/Province Code (e.g., CA, NY, BC):</label>
                <input type="text" id="state_code" name="state_code" required value="CA">
            </div>

            <div class="form-group">
                <label for="region">Region (e.g., SF Bay Area, Greater London):</label>
                <input type="text" id="region" name="region" required value="SF Bay Area">
            </div>

            <div class="form-group">
                <label for="city">City (e.g., San Francisco, New York):</label>
                <input type="text" id="city" name="city" required value="San Francisco">
            </div>

            <div class="form-group">
                <label for="status">Current Status (operating, closed, acquired, ipo):</label>
                <select id="status" name="status" required>
                    <option value="operating">Operating</option>
                    <option value="closed">Closed</option>
                    <option value="acquired">Acquired</option>
                    <option value="ipo">IPO</option>
                </select>
            </div>

            <div class="form-group">
                <label for="funding_rounds">Number of Funding Rounds (e.g., 3):</label>
                <input type="number" id="funding_rounds" name="funding_rounds" required min="1" value="3">
            </div>

            <button type="submit">Predict Success</button>
        </form>

        {% if result %}
        <div class="result-box">
            <div class="result-title">Prediction Result:</div>
            {% if result.class == 1 %}
                <div class="prediction-value" style="color: #28a745;">
                    SUCCESS ({{ result.probability }})
                </div>
                <p style="text-align: center; color: #28a745; font-weight: 600;">The model predicts this startup will be Highly Funded.</p>
            {% else %}
                <div class="prediction-value" style="color: #dc3545;">
                    FAILURE ({{ result.probability }})
                </div>
                <p style="text-align: center; color: #dc3545; font-weight: 600;">The model predicts this startup will not be Highly Funded.</p>
            {% endif %}

            <hr style="border: 0; height: 1px; background: #ddd; margin: 15px 0;">
            <div class="prob-detail">
                <p style="font-weight: 600; text-align: center;">Probability Details:</p>
                <ul>
                    {% for class_name, probability in result.full_probs.items() %}
                    <li>**Class {{ class_name }} ({{ 'Success' if class_name == 1 else 'Failure' }}):** {{ probability }}</li>
                    {% endfor %}
                </ul>
            </div>
        </div>
        {% endif %}

    </div>
</body>
</html>
"""
with open('templates/index.html', 'w') as f:
    f.write(html_code)
print("Created templates/index.html")


# ### Step 2: Upload Model Assets
# You must upload your two model-related files to the root directory of the Colab environment using the file explorer (folder icon on the left):
# 1.  **`model.joblib`** (Your trained classifier)
# 2.  **`features.json`** (The JSON file containing the list of all OHE column names)

# ### Step 3: Run the Flask App
# Run the following cell to start the Flask server and create a public URL using `pyngrok`.

# ```python
# # Note: pyngrok was installed in Step 1.
# import subprocess
# from pyngrok import ngrok

# # Kill any existing ngrok process to free up the tunnel
# !pkill -f ngrok

# # Start Flask app in the background
# process = subprocess.Popen(['python', 'app.py'])

# # Setup ngrok tunnel for port 5000
# public_url = ngrok.connect(5000)
# print(f" * Public URL for your Flask App: {public_url}")

# print("\n--- Model Status ---")
# # The app.py prints success/failure messages upon loading the model,
# # which will appear in the Colab console above this output.

# Click the **Public URL** provided in the output to access your live Startup Success Prediction form!


Dependencies installed (Flask, pandas, scikit-learn, numpy, joblib, pyngrok).
Created app.py
Created templates/index.html


In [126]:
import pandas as pd

FILE_NAME = 'prepared_for_model.csv'

try:
    df = pd.read_csv(FILE_NAME)
    print(f"Successfully loaded data from {FILE_NAME}. Shape: {df.shape}")
except FileNotFoundError:
    print(f"Error: The file '{FILE_NAME}' was not found. Please ensure it exists in the current directory.")
    df = pd.DataFrame() # Initialize an empty DataFrame to avoid errors in subsequent steps
except Exception as e:
    print(f"An error occurred while loading the file: {e}")
    df = pd.DataFrame() # Initialize an empty DataFrame to avoid errors in subsequent steps

Successfully loaded data from prepared_for_model.csv. Shape: (3000, 11)


In [127]:
print("\n--- Modifying Feature Selection and Applying Constraints ---")

# 1. Rename the target column to 'success'
df.rename(columns={'is_highly_funded': 'success'}, inplace=True)
print("Target column 'is_highly_funded' renamed to 'success'.")

# 2. Apply the domain constraint: A closed company cannot be considered successful.
# We explicitly set the 'success' score to 0 for any row where status is 'closed'.
initial_success_count_for_closed = df[(df['status'] == 'closed') & (df['success'] == 1)].shape[0]
df.loc[df['status'] == 'closed', 'success'] = 0
print(f"Applied constraint: Set 'success' to 0 for {initial_success_count_for_closed} rows where status was 'closed' and 'success' was 1.")

# 3. Define the new list of feature columns, replacing 'region' and 'city' with 'funding_total_usd'.
feature_cols = [
    'category_list', 'country_code', 'state_code', 'funding_rounds', 'status', 'funding_total_usd'
]

# 4. Create the features DataFrame (X)
X = df[feature_cols]

# 5. Create the target Series (y)
y = df['success']

print(f"New features selected: {feature_cols}")
print(f"Shape of features (X): {X.shape}")
print(f"Shape of target (y): {y.shape}")
print("Feature selection and constraints applied.")


--- Modifying Feature Selection and Applying Constraints ---
Target column 'is_highly_funded' renamed to 'success'.
Applied constraint: Set 'success' to 0 for 121 rows where status was 'closed' and 'success' was 1.
New features selected: ['category_list', 'country_code', 'state_code', 'funding_rounds', 'status', 'funding_total_usd']
Shape of features (X): (3000, 6)
Shape of target (y): (3000,)
Feature selection and constraints applied.


In [128]:
print("\n--- Feature Engineering: One-Hot Encoding (Converting Text to Numbers) ---")

# Identify categorical columns for one-hot encoding
OHE_COLS = ['category_list', 'country_code', 'state_code', 'status']

# Ensure 'funding_total_usd' is treated as a numerical column and not encoded.
# It's already in X and will be kept as is.

# Apply one-hot encoding to the identified categorical columns in X.
# drop_first=True helps prevent multicollinearity.
X_encoded = pd.get_dummies(X, columns=OHE_COLS, drop_first=True)

print(f"Original feature columns (before encoding): {len(feature_cols)}")
print(f"New encoded feature columns (numbers only): {X_encoded.shape[1]}")
print("Data is now numerical and ready for the model.")


--- Feature Engineering: One-Hot Encoding (Converting Text to Numbers) ---
Original feature columns (before encoding): 6
New encoded feature columns (numbers only): 1531
Data is now numerical and ready for the model.


In [129]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, f1_score
import numpy as np
import joblib
import json

print("\n--- 4. Data Splitting (Train/Test Split) ---")
# We split the data so the model can learn from one part (Train) and be tested
# on the other part (Test) to see if it generalizes to new data.
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training set size: {X_train.shape[0]} rows (80% of data)")
print(f"Testing set size: {X_test.shape[0]} rows (20% of data)")

print("\n--- 5. Hyperparameter Tuning using GridSearchCV ---")
print("This step searches for the optimal parameters to maximize the F1-Score...")

rf_base = RandomForestClassifier(random_state=42, class_weight='balanced')

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, None]
}

grid_search = GridSearchCV(
    estimator=rf_base,
    param_grid=param_grid,
    scoring='f1',
    cv=3,
    verbose=0,
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

final_model = grid_search.best_estimator_

print("Hyperparameter tuning complete.")
print(f"BEST parameters found: {grid_search.best_params_}")

print("\n--- 6. Model Training and Evaluation ---")
# Use the best model from GridSearch as the final model
model = final_model

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
f1 = f1_score(y_test, y_pred)
print(f"Model F1-Score on Test Data: {f1:.4f}")

print("\n--- 7. Saving the Model and Feature Names ---")
# Save the retrained model
joblib.dump(model, 'model.joblib')
print("Retrained model saved to model.joblib")

# Save the feature names
feature_names = X_encoded.columns.tolist()
with open('features.json', 'w') as f:
    json.dump(feature_names, f)
print("Feature names saved to features.json")



--- 4. Data Splitting (Train/Test Split) ---
Training set size: 2400 rows (80% of data)
Testing set size: 600 rows (20% of data)

--- 5. Hyperparameter Tuning using GridSearchCV ---
This step searches for the optimal parameters to maximize the F1-Score...
Hyperparameter tuning complete.
BEST parameters found: {'max_depth': None, 'n_estimators': 50}

--- 6. Model Training and Evaluation ---
Model F1-Score on Test Data: 0.9982

--- 7. Saving the Model and Feature Names ---
Retrained model saved to model.joblib
Feature names saved to features.json


In [130]:
import pandas as pd

FILE_NAME = 'prepared_for_model.csv'

try:
    df = pd.read_csv(FILE_NAME)
    print(f"Successfully loaded data from {FILE_NAME}. Shape: {df.shape}")
except FileNotFoundError:
    print(f"Error: The file '{FILE_NAME}' was not found. Please ensure it exists in the current directory.")
    df = pd.DataFrame() # Initialize an empty DataFrame to avoid errors in subsequent steps
except Exception as e:
    print(f"An error occurred while loading the file: {e}")
    df = pd.DataFrame() # Initialize an empty DataFrame to avoid errors in subsequent steps

Successfully loaded data from prepared_for_model.csv. Shape: (3000, 11)


In [131]:
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix, f1_score
import numpy as np

# 6.1 F1-Score and Classification Report
# F1-Score measures the balance between Precision (how accurate is the '1' prediction?) and
# Recall (how many true '1's did the model find?).
f1 = f1_score(y_test, y_pred)

print(f"Model F1-Score on Test Data: {f1:.4f} (Closer to 1.0 is better)")
print("\nClassification Report (Focus on F1-score for class 1):")
print(classification_report(y_test, y_pred, target_names=['Moderately Funded (0)', 'Highly Funded (1)']))


# 6.2 Confusion Matrix
# This shows exactly where the model was right and wrong on the test data.
cm = confusion_matrix(y_test, y_pred)
# Unpack the matrix components: True Negatives, False Positives, False Negatives, True Positives
tn, fp, fn, tp = cm.ravel()

print("\nConfusion Matrix (Test Data):")
print("------------------------------------------")
print("Interpreting the Matrix:")
print(f"| True Negatives (Correctly guessed 0s): {tn:<10} | False Positives (Wrongly guessed 1s): {fp:<10} |")
print(f"| False Negatives (Missed 1s): {fn:<10} | True Positives (Correctly guessed 1s): {tp:<10} |")
print("------------------------------------------")

# --- 7. Feature Importance (Why the model made the decision) ---
print("\n--- 7. Feature Importance (Top 10) ---")

# Feature importance tells us which input factors (Country, City, Round number, Industry)
# the model considered most important when making a prediction.
feature_importances = pd.Series(model.feature_importances_, index=X_encoded.columns)
top_features = feature_importances.nlargest(10)

print("Most important for predicting success (1):")
print(top_features.to_markdown(numalign="left", stralign="left"))

Model F1-Score on Test Data: 0.9982 (Closer to 1.0 is better)

Classification Report (Focus on F1-score for class 1):
                       precision    recall  f1-score   support

Moderately Funded (0)       1.00      1.00      1.00       324
    Highly Funded (1)       1.00      1.00      1.00       276

             accuracy                           1.00       600
            macro avg       1.00      1.00      1.00       600
         weighted avg       1.00      1.00      1.00       600


Confusion Matrix (Test Data):
------------------------------------------
Interpreting the Matrix:
| True Negatives (Correctly guessed 0s): 324        | False Positives (Wrongly guessed 1s): 0          |
| False Negatives (Missed 1s): 1          | True Positives (Correctly guessed 1s): 275        |
------------------------------------------

--- 7. Feature Importance (Top 10) ---
Most important for predicting success (1):
|                                   | 0          |
|:----------------------

In [132]:
import pandas as pd

FILE_NAME = 'prepared_for_model.csv'

try:
    df = pd.read_csv(FILE_NAME)
    print(f"Successfully loaded data from {FILE_NAME}. Shape: {df.shape}")
except FileNotFoundError:
    print(f"Error: The file '{FILE_NAME}' was not found. Please ensure it exists in the current directory.")
    df = pd.DataFrame() # Initialize an empty DataFrame to avoid errors in subsequent steps
except Exception as e:
    print(f"An error occurred while loading the file: {e}")
    df = pd.DataFrame() # Initialize an empty DataFrame to avoid errors in subsequent steps

Successfully loaded data from prepared_for_model.csv. Shape: (3000, 11)


In [133]:
print("\n--- Modifying Feature Selection and Applying Constraints ---")

# 1. Rename the target column to 'success'
df.rename(columns={'is_highly_funded': 'success'}, inplace=True)
print("Target column 'is_highly_funded' renamed to 'success'.")

# 2. Apply the domain constraint: A closed company cannot be considered successful.
# We explicitly set the 'success' score to 0 for any row where status is 'closed'.
initial_success_count_for_closed = df[(df['status'] == 'closed') & (df['success'] == 1)].shape[0]
df.loc[df['status'] == 'closed', 'success'] = 0
print(f"Applied constraint: Set 'success' to 0 for {initial_success_count_for_closed} rows where status was 'closed' and 'success' was 1.")

# 3. Define the new list of feature columns, replacing 'region' and 'city' with 'funding_total_usd'.
feature_cols = [
    'category_list', 'country_code', 'state_code', 'funding_rounds', 'status', 'funding_total_usd'
]

# 4. Create the features DataFrame (X)
X = df[feature_cols]

# 5. Create the target Series (y)
y = df['success']

print(f"New features selected: {feature_cols}")
print(f"Shape of features (X): {X.shape}")
print(f"Shape of target (y): {y.shape}")
print("Feature selection and constraints applied.")


--- Modifying Feature Selection and Applying Constraints ---
Target column 'is_highly_funded' renamed to 'success'.
Applied constraint: Set 'success' to 0 for 121 rows where status was 'closed' and 'success' was 1.
New features selected: ['category_list', 'country_code', 'state_code', 'funding_rounds', 'status', 'funding_total_usd']
Shape of features (X): (3000, 6)
Shape of target (y): (3000,)
Feature selection and constraints applied.


In [134]:
print("\n--- Feature Engineering: One-Hot Encoding (Converting Text to Numbers) ---")

# Identify categorical columns for one-hot encoding
OHE_COLS = ['category_list', 'country_code', 'state_code', 'status']

# Ensure 'funding_total_usd' is treated as a numerical column and not encoded.
# It's already in X and will be kept as is.

# Apply one-hot encoding to the identified categorical columns in X.
# drop_first=True helps prevent multicollinearity.
X_encoded = pd.get_dummies(X, columns=OHE_COLS, drop_first=True)

print(f"Original feature columns (before encoding): {len(feature_cols)}")
print(f"New encoded feature columns (numbers only): {X_encoded.shape[1]}")
print("Data is now numerical and ready for the model.")


--- Feature Engineering: One-Hot Encoding (Converting Text to Numbers) ---
Original feature columns (before encoding): 6
New encoded feature columns (numbers only): 1531
Data is now numerical and ready for the model.


In [135]:
import pandas as pd

# Define the file name from the previous preprocessing step
FILE_NAME = 'prepared_for_model.csv'

# --- 1. Load Data to extract unique values for dropdowns ---
try:
    df_for_dropdowns = pd.read_csv(FILE_NAME)
    print(f"Successfully loaded data from {FILE_NAME} for dropdown value extraction. Shape: {df_for_dropdowns.shape}")
except FileNotFoundError:
    print(f"ERROR: The file '{FILE_NAME}' was not found.")
    print("Please ensure you have run 'data_preprocessing_eda.py' first.")
    exit()
except Exception as e:
    print(f"An error occurred while reading the file: {e}")
    exit()

# Extract unique, sorted values for dropdowns
unique_categories = sorted(df_for_dropdowns['category_list'].dropna().unique().tolist())
unique_countries = sorted(df_for_dropdowns['country_code'].dropna().unique().tolist())
unique_states = sorted(df_for_dropdowns['state_code'].dropna().unique().tolist())
unique_statuses = sorted(df_for_dropdowns['status'].dropna().unique().tolist())

print(f"\nExtracted {len(unique_categories)} unique categories.")
print(f"Extracted {len(unique_countries)} unique countries.")
print(f"Extracted {len(unique_states)} unique states.")
print(f"Extracted {len(unique_statuses)} unique statuses.")

Successfully loaded data from prepared_for_model.csv for dropdown value extraction. Shape: (3000, 11)

Extracted 1380 unique categories.
Extracted 19 unique countries.
Extracted 130 unique states.
Extracted 4 unique statuses.


In [136]:
import os
import subprocess
from IPython.display import clear_output

# --- Write app.py (The Flask Application) ---
app_code = """
import json
import joblib
import pandas as pd
import numpy as np
from flask import Flask, request, render_template, redirect, url_for

# --- Configuration and Initialization ---
app = Flask(__name__)
MODEL_PATH = 'model.joblib'
FEATURES_PATH = 'features.json'
FILE_NAME = 'prepared_for_model.csv' # Added for loading data in app.py

# Global variables to hold the loaded model and feature list
model = None
feature_names = None

# Global variables for dropdown options, loaded once
unique_categories = []
unique_countries = []
unique_states = []
unique_statuses = []

# List of the original categorical columns that need OHE
OHE_COLS = ['category_list', 'country_code', 'state_code', 'status']

def load_assets():
    # Load the trained model and feature names list.
    global model, feature_names
    global unique_categories, unique_countries, unique_states, unique_statuses # Declare globals

    # 1. Load the Model
    try:
        model = joblib.load(MODEL_PATH)
        print("✅ Model loaded successfully.")
    except FileNotFoundError:
        print(f"❌ Error: Model file not found at {MODEL_PATH}. Ensure you have uploaded 'model.joblib'.")
        model = None
    except Exception as e:
        print(f"❌ Error loading model: {e}")
        model = None

    # 2. Load the Feature Names (from training data)
    try:
        with open(FEATURES_PATH, 'r') as f:
            feature_names = json.load(f)
        print(f"✅ Features list loaded successfully. Total OHE features: {len(feature_names)}")
    except FileNotFoundError:
        print(f"❌ Error: Features file not found at {FEATURES_PATH}. Ensure you have uploaded 'features.json'.")
        feature_names = []
    except Exception as e:
        print(f"❌ Error loading features: {e}")
        feature_names = []

    # 3. Load data for dropdowns (this will run once when app starts)
    try:
        df_for_dropdowns = pd.read_csv(FILE_NAME)
        unique_categories = sorted(df_for_dropdowns['category_list'].dropna().unique().tolist())
        unique_countries = sorted(df_for_dropdowns['country_code'].dropna().unique().tolist())
        unique_states = sorted(df_for_dropdowns['state_code'].dropna().unique().tolist())
        unique_statuses = sorted(df_for_dropdowns['status'].dropna().unique().tolist())
        print(f"✅ Dropdown values loaded successfully from '{FILE_NAME}'. Found {len(unique_categories)} categories, {len(unique_countries)} countries, {len(unique_states)} states, {len(unique_statuses)} statuses.")
    except FileNotFoundError:
        print(f"❌ Error: '{FILE_NAME}' not found for dropdowns. Please ensure it exists.")
    except Exception as e:
        print(f"❌ Error loading dropdown data: {e}")

load_assets()

@app.route('/')
def cover():
    return render_template('cover.html')

@app.route('/intro')
def intro():
    return render_template('intro.html')

@app.route('/predict', methods=['GET', 'POST'])
def predict():
    # Renders the main form and handles prediction on POST request.

    if model is None or not feature_names:
        return render_template('predict.html', error="Model assets are missing or failed to load. Please check the console output.")

    prediction_result = None
    error_message = None

    if request.method == 'POST':
        try:
            raw_input = {
                'category_list': request.form['category_list'],
                'country_code': request.form['country_code'],
                'state_code': request.form['state_code'],
                'funding_rounds': int(request.form['funding_rounds']),
                'status': request.form['status'],
                'funding_total_usd': float(request.form['funding_total_usd']) # New direct numeric input
            }

            input_df = pd.DataFrame([raw_input])

            # Apply one-hot encoding to categorical features
            input_encoded = pd.get_dummies(input_df, columns=OHE_COLS, drop_first=False)

            # Reindex to match the columns from training data, filling missing with 0
            # Ensure numerical columns are not accidentally encoded or dropped
            X_new = input_encoded.reindex(columns=feature_names, fill_value=0)

            prediction_class = model.predict(X_new)[0]

            probabilities = model.predict_proba(X_new)[0]
            class_labels = model.classes_
            prob_map = dict(zip(class_labels, probabilities))

            predicted_probability = prob_map[prediction_class] * 100

            prediction_result = {
                'class': int(prediction_class),
                'probability': f"{predicted_probability:.2f}%",
                'full_probs': {int(k): f"{v*100:.2f}%" for k, v in prob_map.items()}
            }

        except ValueError:
            error_message = "Invalid input. Please ensure 'Funding Rounds' and 'Funding Total (USD)' are numbers and all fields are filled."
        except Exception as e:
            print(f"An unexpected error occurred during prediction: {e}")
            error_message = f"Prediction failed due to an internal error: {e}"

    return render_template(
        'predict.html',
        unique_categories=unique_categories,
        unique_countries=unique_countries,
        unique_states=unique_states,
        unique_statuses=unique_statuses,
        prediction_result=prediction_result,
        error=error_message,
        form_values=request.form
    )

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)
"""
with open('app.py', 'w') as f:
    f.write(app_code)
print("Created app.py")


# --- Write templates/cover.html ---
cover_html_code = """
<!DOCTYPE html>
<html lang=\"en\">
<head>
    <meta charset=\"UTF-8\">
    <meta name=\"viewport\" content=\"width=device-width, initial-scale=1.0\">
    <title>Cover Page - Startup Success Prediction</title>
    <style>
        @import url('https://fonts.googleapis.com/css2?family=Inter:wght@400;600;700;800&display=swap');
        body {
            font-family: 'Inter', sans-serif;
            background: linear-gradient(135deg, #f0f4f8, #d9e2ec); /* Soft gradient background */
            color: #2c3e50;
            display: flex;
            justify-content: center;
            align-items: center;
            min-height: 100vh;
            margin: 0;
            overflow: hidden; /* Hide potential scrollbars */
        }
        .cover-container {
            background-color: #ffffff; /* White background for content */
            padding: 50px 60px;
            border-radius: 20px;
            box-shadow: 0 10px 30px rgba(0, 0, 0, 0.15); /* Stronger shadow for depth */
            text-align: center;
            max-width: 700px;
            width: 90%;
            animation: fadeIn 1.5s ease-out; /* Fade-in animation */
        }
        @keyframes fadeIn {
            from { opacity: 0; transform: translateY(20px); }
            to { opacity: 1; transform: translateY(0); }
        }
        h1 {
            color: #007bff; /* Primary blue color */
            font-size: 3.5em;
            margin-bottom: 15px;
            font-weight: 800;
            letter-spacing: -1px;
        }
        p {
            font-size: 1.2em;
            line-height: 1.7;
            margin-bottom: 30px;
            color: #5f728c;
        }
        .start-button {
            display: inline-block;
            padding: 15px 35px;
            background-color: #28a745; /* Success green */
            color: white;
            text-decoration: none;
            border-radius: 30px;
            font-size: 1.3em;
            font-weight: 700;
            transition: background-color 0.3s ease, transform 0.2s ease;
            box-shadow: 0 5px 15px rgba(40, 167, 69, 0.4);
        }
        .start-button:hover {
            background-color: #218838;
            transform: translateY(-3px); /* Lift effect on hover */
        }
        .start-button:active {
            transform: translateY(-1px); /* Press effect */
        }
        .footer-nav {
            margin-top: 40px;
            font-size: 0.9em;
            color: #999;
        }
        .footer-nav a {
            color: #007bff;
            text-decoration: none;
            margin: 0 10px;
            font-weight: 600;
        }
        .footer-nav a:hover {
            text-decoration: underline;
        }
    </style>
</head>
<body>
    <div class=\"cover-container\">
        <h1>Startup Success Predictor</h1>
        <p>Uncover the potential of your startup idea with our AI-powered prediction model. Discover the factors that contribute to a startup being highly funded.</p>
        <a href=\"/intro\" class=\"start-button\">Get Started</a>
        <div class=\"footer-nav\">
            &copy; 2024 Startup Predictor. All rights reserved.
        </div>
    </div>
</body>
</html>
"""
with open('templates/cover.html', 'w') as f:
    f.write(cover_html_code)
print("Created templates/cover.html")


# --- Write templates/intro.html ---
intro_html_code = """
<!DOCTYPE html>
<html lang=\"en\">
<head>
    <meta charset=\"UTF-8\">
    <meta name=\"viewport\" content=\"width=device-width, initial-scale=1.0\">
    <title>Introduction - Startup Success Prediction</title>
    <style>
        @import url('https://fonts.googleapis.com/css2?family=Inter:wght@400;600;700&display=swap');
        body {
            font-family: 'Inter', sans-serif;
            background-color: #f8f9fa;
            color: #333;
            display: flex;
            justify-content: center;
            align-items: flex-start;
            padding-top: 40px;
            margin: 0;
            min-height: 100vh;
        }
        .container {
            background-color: #fff;
            padding: 30px 40px;
            border-radius: 12px;
            box-shadow: 0 4px 15px rgba(0, 0, 0, 0.1);
            width: 100%;
            max-width: 600px;
        }
        h1 {
            text-align: center;
            color: #007bff;
            margin-bottom: 30px;
            font-weight: 700;
        }
        p {
            line-height: 1.6;
            margin-bottom: 15px;
        }
        .button-link {
            display: block;
            width: fit-content;
            margin: 20px auto 0;
            padding: 12px 25px;
            background-color: #28a745;
            color: white;
            text-align: center;
            text-decoration: none;
            border-radius: 8px;
            font-size: 16px;
            font-weight: 600;
            transition: background-color 0.3s, transform 0.1s;
        }
        .button-link:hover {
            background-color: #218838;
            transform: scale(1.02);
        }
        .button-link:active {
            transform: translateY(-1px);
        }
        .nav-links {
            text-align: center;
            margin-top: 30px;
            padding-top: 20px;
            border-top: 1px solid #eee;
        }
        .nav-links a {
            color: #007bff;
            text-decoration: none;
            margin: 0 10px;
            font-weight: 600;
        }
        .nav-links a:hover {
            text-decoration: underline;
        }
    </style>
</head>
<body>
    <div class=\"container\">
        <h1>Welcome to the Startup Success Predictor!</h1>
        <p>This application uses a Machine Learning model to predict the likelihood of a startup being 'Highly Funded' based on various attributes.</p>
        <p>The model was trained on a dataset containing information about numerous startups, including their industry, location, funding rounds, and total funding received. 'Highly Funded' is defined as startups that have received total funding above the median funding amount in our cleaned dataset.</p>
        <p>Use this tool to explore how different characteristics might influence a startup's funding outcome.</p>
        <div class=\"nav-links\">
            <a href=\"/\">Back</a> |
            <a href=\"/predict\">Next</a>
        </div>
    </div>
</body>
</html>
"""
with open('templates/intro.html', 'w') as f:
    f.write(intro_html_code)
print("Created templates/intro.html")


# --- Write templates/predict.html (Renamed from index.html) ---
predict_html_code = """
<!DOCTYPE html>
<html lang=\"en\">
<head>
    <meta charset=\"UTF-8\">
    <meta name=\"viewport\" content=\"width=device-width, initial-scale=1.0\">
    <title>Startup Success Prediction</title>
    <style>
        @import url('https://fonts.googleapis.com/css2?family=Inter:wght@400;600;700&display=swap');
        body {
            font-family: 'Inter', sans-serif;
            background-color: #f8f9fa;
            color: #333;
            display: flex;
            justify-content: center;
            align-items: flex-start;
            padding-top: 40px;
            margin: 0;
            min-height: 100vh;
        }
        .container {
            background-color: #fff;
            padding: 30px 40px;
            border-radius: 12px;
            box-shadow: 0 4px 15px rgba(0, 0, 0, 0.1);
            width: 100%;
            max-width: 600px;
        }
        h1 {
            text-align: center;
            color: #007bff;
            margin-bottom: 30px;
            font-weight: 700;
        }
        .form-group {
            margin-bottom: 20px;
        }
        label {
            display: block;
            margin-bottom: 8px;
            font-weight: 600;
            color: #555;
        }
        input[type=\"number\"], input[type=\"text\"], select {
            width: 100%;
            padding: 10px;
            border: 1px solid #ccc;
            border-radius: 8px;
            box-sizing: border-box;
            transition: border-color 0.3s;
        }
        input[type=\"number\"]:focus, input[type=\"text\"]:focus, select:focus {
            border-color: #007bff;
            outline: none;
        }
        button {
            width: 100%;
            padding: 12px;
            background-color: #28a745;
            color: white;
            border: none;
            border-radius: 8px;
            font-size: 16px;\n            cursor: pointer;\n            transition: background-color 0.3s, transform 0.1s;\n            font-weight: 600;\n        }\n        button:hover {\n            background-color: #218838;\n        }\n        button:active {\n            transform: scale(0.99);\n        }\n        .result-box {\n            margin-top: 30px;\n            padding: 20px;\n            border-radius: 8px;\n            background-color: #e9f7ef;\n            border: 1px solid #c3e6cb;\n        }\n        .result-title {\n            font-size: 1.2em;\n            font-weight: 700;\n            color: #28a745;\n            margin-bottom: 10px;\n            text-align: center;\n        }\n        .prediction-value {\n            font-size: 2em;\n            font-weight: 800;\n            text-align: center;\n            color: #007bff;\n            margin-bottom: 15px;\n        }\n        .prob-detail {\n            font-size: 0.9em;\n            color: #555;\n            line-height: 1.5;\n        }\n        .error-message {\n            margin-top: 20px;\n            padding: 15px;\n            background-color: #f8d7da;\n            color: #721c24;\n            border: 1px solid #f5c6cb;\n            border-radius: 8px;\n            text-align: center;\n        }\n    </style>\n</head>\n<body>\n    <div class=\"container\">\n        <h1>Startup Success Prediction</h1>\n        <p style=\"text-align: center; color: #666;\">Predicts the likelihood of success (Class 1) based on key startup attributes.</p>\n
        {% if error %}\n        <div class=\"error-message\">{{ error }}</div>\n        {% endif %}\n
        <form method=\"POST\" action=\"/predict\">\n
            <div class=\"form-group\">\n                <label for=\"category_list\">Industry Category:</label>\n                <select id=\"category_list\" name=\"category_list\" required>\n                    {% for cat in unique_categories %}\n                        <option value=\"{{ cat }}\" {% if form_values.category_list == cat %}selected{% endif %}>{{ cat }}</option>\n                    {% endfor %}\n                </select>\n            </div>\n
            <div class=\"form-group\">\n                <label for=\"country_code\">Country Code:</label>\n                <select id=\"country_code\" name=\"country_code\" required>\n                    {% for country in unique_countries %}\n                        <option value=\"{{ country }}\" {% if form_values.country_code == country %}selected{% endif %}>{{ country }}</option>\n                    {% endfor %}\n                </select>\n            </div>\n
            <div class=\"form-group\">\n                <label for=\"state_code\">State/Province Code:</label>\n                <select id=\"state_code\" name=\"state_code\" required>\n                    {% for state in unique_states %}\n                        <option value=\"{{ state }}\" {% if form_values.state_code == state %}selected{% endif %}>{{ state }}</option>\n                    {% endfor %}\n                </select>\n            </div>\n
            <div class=\"form-group\">\n                <label for=\"funding_total_usd\">Total Funding in USD:</label>\n                <input type=\"number\" id=\"funding_total_usd\" name=\"funding_total_usd\" required step=\"any\" value=\"{{ form_values.funding_total_usd | default('1000000.0') }}\">\n            </div>\n
            <div class=\"form-group\">\n                <label for=\"status\">Current Status:</label>\n                <select id=\"status\" name=\"status\" required>\n                    {% for stat in unique_statuses %}\n                        <option value=\"{{ stat }}\" {% if form_values.status == stat %}selected{% endif %}>{{ stat }}</option>\n                    {% endfor %}\n                </select>\n            </div>\n
            <div class=\"form-group\">\n                <label for=\"funding_rounds\">Number of Funding Rounds:</label>\n                <input type=\"number\" id=\"funding_rounds\" name=\"funding_rounds\" required min=\"1\" value=\"{{ form_values.funding_rounds | default('3') }}\">\n            </div>\n
            <button type=\"submit\">Predict Success</button>\n        </form>\n
        {% if prediction_result %}\n        <div class=\"result-box\">\n            <div class=\"result-title\">Prediction Result:</div>\n            {% if prediction_result.class == 1 %}\n                <div class=\"prediction-value\" style=\"color: #28a745;\">\n                    SUCCESS ({{ prediction_result.probability }})\n                </div>\n                <p style=\"text-align: center; color: #28a745; font-weight: 600;\">The model predicts this startup will be Highly Funded.</p>\n            {% else %}\n                <div class=\"prediction-value\" style=\"color: #dc3545;\">\n                    FAILURE ({{ prediction_result.probability }})\n                </div>\n                <p style=\"text-align: center; color: #dc3545; font-weight: 600;\">The model predicts this startup will not be Highly Funded.</p>\n            {% endif %}\n
            <hr style=\"border: 0; height: 1px; background: #ddd; margin: 15px 0;\">\n            <div class=\"prob-detail\">\n                <p style=\"font-weight: 600; text-align: center;\">Probability Details:</p>\n                <ul>\n                    {% for class_name, probability in prediction_result.full_probs.items() %}\n                    <li>**Class {{ class_name }} ({{ 'Success' if class_name == 1 else 'Failure' }}):** {{ probability }}</li>\n                    {% endfor %}\n                </ul>\n            </div>\n        </div>\n        {% endif %}\n
        <div class=\"nav-links\">
            <a href=\"/\">Cover Page</a> |
            <a href=\"/intro\">Introduction</a>
        </div>

    </div>\n</body>\n</html>\n"""
with open('templates/predict.html', 'w') as f:
    f.write(predict_html_code)
print("Created templates/predict.html (renamed from index.html)")


Created app.py
Created templates/cover.html
Created templates/intro.html
Created templates/predict.html (renamed from index.html)


In [137]:
import subprocess
from pyngrok import ngrok
import time

# !!! IMPORTANT: Replace 'YOUR_AUTH_TOKEN' with your actual ngrok authtoken !!!
# You can get your authtoken from https://dashboard.ngrok.com/get-started/your-authtoken
ngrok.set_auth_token('35Cfdio6TEgObKq0FaQIjHW5pKy_6bD285niZUsJA8xDwaNJL') # Make sure to put your actual token here

# Kill any existing ngrok process to free up the tunnel
!pkill -f ngrok

# Start Flask app in the background, capturing stdout/stderr
process = subprocess.Popen(['python', 'app.py'], stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

# Give Flask a moment to start and print its initial messages
time.sleep(5) # Give it 5 seconds to print startup messages

print("\n--- Flask App Startup Logs (Initial) ---")
# Read and print the initial lines from the Flask app's output
# This might not capture all logs if Flask is very verbose or slow to start,
# but should catch the "Model loaded" messages.
for _ in range(10): # Read up to 10 lines of initial output
    line = process.stdout.readline()
    if line:
        print(line.strip())
    else:
        break
print("--- End Flask App Startup Logs ---")

# Setup ngrok tunnel for port 5000
public_url = ngrok.connect(5000)
print(f" * Public URL for your Flask App: {public_url}")


--- Flask App Startup Logs (Initial) ---
/usr/local/lib/python3.12/dist-packages/sklearn/base.py:442: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.6.1 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/base.py:442: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.6.1 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
warnings.warn(
✅ Model loaded successfully.
✅ Features list loaded successfully. Total OHE features: 1531
✅ Dropdown values loaded successfully from 'prepared_for_model.csv'. 